In [1]:
##################### Importing the libraries ##########################
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
import datetime as dt
import talib as ta
import ffn

In [2]:
##################### Downloading the data ################################
end1 = dt.date(2022, 5, 30) 
start1 = dt.date(2012, 5, 30)

df = yf.download("HDFCBANK.NS", start=start1, end=end1)
df.dropna(inplace =True)

[*********************100%***********************]  1 of 1 completed


In [3]:
df.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-05-30,250.024994,252.699997,248.274994,250.324997,231.645004,5083640
2012-05-31,248.000000,254.449997,247.675003,253.100006,234.212952,9519560
2012-06-01,252.000000,253.574997,245.000000,245.199997,226.902466,5854374
2012-06-04,244.000000,249.250000,242.524994,247.824997,229.331558,5680484
2012-06-05,250.000000,252.449997,249.225006,250.750000,232.038300,5355356


In [4]:
################## Main loop #############################
TradeBook = pd.DataFrame(columns=['LBperiod','Total_P&L','Total_Returns','Total_Trade',\
                                 'Calmar_ratio','Drawdown','Sharpe_ratio'])
Openvaluedict = dict()
ema_values = range(5,250) 
for i in ema_values:
    Openvaluedict = {'LBperiod':None,'Total_P&L':None,'Total_Returns':None,'Total_Trade':None,\
                      'Calmar_ratio':None,'Drawdown':None,'Sharpe_ratio':None}
    Openvaluedict['LBperiod']=i
    ################# Calculating the EMA #######################
    df['Ema_high'] = df['High'].ewm(span=i, adjust=False).mean()
    df['Ema_low'] = df['Low'].ewm(span=i, adjust=False).mean()

    ################### buy and sell condition ##################
    df['Cc_returns'] = df['Close'] - (df['Close'].shift(1))zzz
    df['Condition1']=np.where((df['Close'] > df['Ema_high']),1,0)
    df['Condition2']=np.where((df['Close'] > df['Open']),1,0)
    df['Condition3']=np.where((df['Close'] < df['Ema_low']),-1,0)
    df['Condition4']=np.where((df['Close'] < df['Open']),-1,0)
    ################### Signal,Position and return ###########################
    df['Long']=np.where(((df['Condition1']==1) & (df['Condition2']==1)),1,0)
    df['Short']=np.where(((df['Condition3']==-1) & (df['Condition4']==-1)),-1,0)
    df['Signal'] =(df['Long']+df['Short'])
    df['Position'] = df['Signal'].replace(to_replace=0, method='ffill')
    df['Position1']=df['Position'].shift(1)
    df['Count']=np.where((df['Position']+df['Position1'])==0,1,0)
    
    df['P&L']= df['Cc_returns'] * df['Position1']
    df['Strategy_returns']=((df['Close']/(df['Close'].shift(1)))-1) *df['Position1']

    ####### Creating a trading account name 'Equity' with a starting capital of 100 ############
    df['Equity']=df['Strategy_returns'].cumsum()+100
    df.dropna(inplace=True)
    
    ########### Total P&L####################
    Openvaluedict['Total_P&L']=(df['P&L'].cumsum()[-1])
    
    ############ Total Returns #####################
    Openvaluedict['Total_Returns']=(df['Strategy_returns'].cumsum()[-1])
    

    ############ Total Trade ######################

    Openvaluedict['Total_Trade'] = df["Count"].sum()
    

    ############# Calmar ratio #####################
    Openvaluedict['Calmar_ratio']=ffn.core.calc_calmar_ratio(df['Equity'])

    ############# Drawdown #########################
    Openvaluedict['Drawdown']=ffn.core.calc_max_drawdown(df['Equity'])

    ############## Sharpe Ratio ####################
    a=(df['P&L'].mean())
    b=df['P&L'].std()
    Openvaluedict['Sharpe_ratio'] = (a/b)*(np.sqrt(250))

    # Record the trade
    TradeBook.loc[len(TradeBook)] = [Openvaluedict['LBperiod'],Openvaluedict['Total_P&L'],\
                                     Openvaluedict['Total_Returns'],Openvaluedict['Total_Trade']\
                                     ,Openvaluedict['Calmar_ratio']\
                                     ,Openvaluedict['Drawdown'],Openvaluedict['Sharpe_ratio']]
    # Reset the OpenPosition Dict
    Openvaluedict = Openvaluedict.fromkeys(Openvaluedict, None)

        

#Openvaluedict[key] = Openvaluedict

In [5]:
TradeBook

,LBperiod,Total_P&L,Total_Returns,Total_Trade,Calmar_ratio,Drawdown,Sharpe_ratio
0,5.0,-358.398880,-0.111278,240.0,-0.014629,-0.007620,-0.159776
1,6.0,-576.849136,-0.303905,235.0,-0.042693,-0.007139,-0.257235
2,7.0,-504.699463,-0.119595,222.0,-0.016070,-0.007463,-0.225122
3,8.0,-734.449524,-0.325100,216.0,-0.042387,-0.007701,-0.327679
4,9.0,-872.749817,-0.396113,210.0,-0.043795,-0.009087,-0.389529
...,...,...,...,...,...,...,...
240,245.0,-480.274628,-0.125132,45.0,-0.015691,-0.008856,-0.226177
241,246.0,-470.374603,-0.094810,44.0,-0.011894,-0.008853,-0.221563
242,247.0,-470.374603,-0.094810,43.0,-0.011905,-0.008853,-0.221613
243,248.0,-418.674622,-0.024638,41.0,-0.003096,-0.008847,-0.197300


In [6]:
TradeBook.to_excel('TradeBook_HDFC.xlsx',index=None)